# 读写文件  
xarray支持多种文件格式的直接序列化和I/O，从简单的Pickle文件到更灵活的netCDF 格式（推荐）。

In [3]:
    import numpy as np
    import pandas as pd
    import xarray as xr

    np.random.seed(123456)

# netCDF

## 文件简述

推荐的存储xarray数据结构的方法是[netCDF](https://www.unidata.ucar.edu/software/netcdf/)，这是一种二进制文件格式，用于自描述于地球科学的数据集。xarray基于netCDF数据模型，因此磁盘上的netCDF文件直接对应于[Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset)对象（更准确地说，netCDF文件中的组直接对应于to Dataset对象。有关更多信息，请参见[group](http://xarray.pydata.org/en/stable/io.html#io-netcdf-groups)。）

NetCDF几乎在所有平台上都受支持，并且存在针对大多数科学编程语言的解析器。netCDF的最新版本基于更广泛使用的HDF5文件格式。

*  如果您不熟悉这种数据格式，那么[netCDF FAQ](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#What-Is-netCDF)是一个不错的起点。

使用xarray读取和写入netCDF文件需要安装scipy或 netCDF4-Python库（读取/写入netCDF V4文件并使用下面描述的压缩选项需要后者）。

## 读写方式

### 创建一个Dataset并写入磁盘

我们可以使用以下[```Dataset.to_netcdf()```](https://github.com/pydata/xarray/blob/master/doc/io.rst#id13)方法将数据集保存到磁盘 ：

In [4]:
   ds_test = xr.Dataset(
        {"foo": (("x", "y"), np.random.rand(4, 5))},
        coords={
            "x": [10, 20, 30, 40],
            "y": pd.date_range("2000-01-01", periods=5),
            "z": ("x", list("abcd")),
        },
    )

In [5]:
ds = ds_test # 试试是不是命名的问题

In [31]:
# ds.to_netcdf('saved_on_disk.nc') #会出现语法错误

In [9]:
ds.to_netcdf('saved_on_disk.nc')

默认情况下，文件另存为netCDF4（假设已安装netCDF4-Python）。您可以使用format和engine参数控制用于写入文件的格式和engine。

通过传递engine='h5netcdf'open_dataset()engine='netcdf4'给[h5netcdf](https://github.com/shoyer/h5netcdf)软件包使用，有时可能比使用netCDF4软件包的默认方法更快。

### 从磁盘读取，及一些特性

我们可以使用以下```open_dataset()```方法加载netCDF文件以创建新的数据集 ：

In [10]:
    ds_disk = xr.open_dataset("saved_on_disk.nc")
    ds_disk

<xarray.Dataset>
Dimensions:  (x: 4, y: 5)
Coordinates:
  * x        (x) int64 10 20 30 40
  * y        (y) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-01-04 2000-01-05
    z        (x) object ...
Data variables:
    foo      (x, y) float64 ...

同样，可以使用[```DataArray.to_netcdf()```](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.to_netcdf.html#xarray.DataArray.to_netcdf)方法将DataArray保存到磁盘，并使用[```open_dataarray()```](http://xarray.pydata.org/en/stable/generated/xarray.open_dataarray.html#xarray.open_dataarray)函数从磁盘加载DataArray 。由于的netCDF文件对应Dataset的对象，这些功能在保存前，从内部转换DataArray为Dataset，然后再在加载时转换回来，确保DataArray所加载总是所保存的那个。

数据总是从netCDF文件中延迟加载。您可以操作，切片和子集化Dataset和DataArray对象，并且在尝试执行某种实际计算之前，不会将任何数组值加载到内存中。有关这些延迟数组如何工作的示例，请参见下面的OPeNDAP部分。

*  重要的是要注意，当您修改数据集的值时，即使是一个链接到磁盘上文件的值，也只会修改您在xarray中操作的内存中副本：永远不会触摸磁盘上的原始文件。

*   xarray延迟加载远程或磁盘数据集通常是但并非总是希望的。在执行计算密集型操作之前，通常最好通过调用Dataset.load()方法将Dataset（或DataArray）完全加载到内存中。

Dataset具有Dataset.close()关闭关联的netCDF文件的方法。但是，使用以下with语句通常更清洁：

In [11]:
# 使用后自动关闭dataset
with xr.open_dataset("saved_on_disk.nc") as ds:
    print(ds.keys())

KeysView(<xarray.Dataset>
Dimensions:  (x: 4, y: 5)
Coordinates:
  * x        (x) int64 10 20 30 40
  * y        (y) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-01-04 2000-01-05
    z        (x) object ...
Data variables:
    foo      (x, y) float64 ...)


尽管xarray为磁盘上的文件的增量读取提供了合理的支持，但它不支持增量写入（?incremental writes），（ 增量写入 这对于处理太大而内存无法容纳的Dataset可能是一种有用的策略）。相反，xarray与dask.array相集成（请参阅使[用Dask进行并行计算](http://xarray.pydata.org/en/stable/dask.html#dask)），后者为流计算提供了功能全面的引擎。

可以使用```mode='a'```参数附加或覆盖netCDF变量。使用此选项时，数据集中的所有变量将被写入原始的netCDF文件，无论它们是否存在于原始数据集中。

## group

数据集也可以加载或写入netCDF文件中的特定group。要从组中加载，请将关键字参数```group```传递给```open_dataset```函数。  
*  可以将group指定为类似路径的字符串:例如，要访问组'foo'中的子组'bar'，则将'/ foo / bar'作为group参数。当在一个文件中写入多个组，将```mode='a'```传递给```to_netcdf```确保每次调用不会删除该文件。

## 读取编码数据

NetCDF文件遵循一些约定来对日期时间数组进行编码（如带有“ units”属性的数字）以及对数据进行打包和拆包（如“ scale_factor”和“ add_offset”属性所述）。如果给定函数```open_dataset()```的参数 decode_cf=True（默认值），则xarray将尝试根据CF约定，自动解码netCDF对象中的值 。有时，这将失败，例如，如果变量具有无效的“units”或“calendar”属性。对于这些情况，您可以手动关闭此解码。

您可以在```DataArray.encoding```和 ```DataArray.encoding```属性中查看此编码信息（以及其他信息） ：

In [39]:
ds

<xarray.Dataset>
Dimensions:  (x: 4, y: 5)
Coordinates:
  * x        (x) int64 10 20 30 40
  * y        (y) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-01-04 2000-01-05
    z        (x) object ...
Data variables:
    foo      (x, y) float64 ...

In [12]:
ds_disk["y"].encoding

{'zlib': False,
 'shuffle': False,
 'complevel': 0,
 'fletcher32': False,
 'contiguous': True,
 'chunksizes': None,
 'source': '/mnt/c/test/python/lx_xarray/saved_on_disk.nc',
 'original_shape': (5,),
 'dtype': dtype('int64'),
 'units': 'days since 2000-01-01 00:00:00',
 'calendar': 'proleptic_gregorian'}

In [13]:
ds_disk.encoding

{'unlimited_dims': set(),
 'source': '/mnt/c/test/python/lx_xarray/saved_on_disk.nc'}

请注意，所有操作除索引之外的变量的操作都将删除编码信息。

In [14]:
ds_disk.close()

## 读取多文件的数据集

### mfdataset

NetCDF文件经常遇到集合多个文件的情况，例如，对应于不同模型运行的不同文件或每个时间戳的一个文件。xarray可以通过 函数```concat()```，```merge()```，```combine_nested()```和 ```combine_by_coords()```直截了当的文件合并成一个单一数据集。有关这些功能之间的区别的详细信息，请参见[合并数据](http://xarray.pydata.org/en/stable/combining.html#combining-data)。
  
Xarray通过dask，包括了用于操纵，不适合到内存中的数据集的一些支持。如果您安装了dask，则可以使用```open_mfdataset()```同时并行打开多个文件：

先创建多个文件（具有不同pd时间序列）(注意ds的时间坐标是'y')：

In [51]:
ds_time_test = pd.date_range("2000-01-01", periods=5*10)

In [58]:
ds_time_test[5:10]

DatetimeIndex(['2000-01-06', '2000-01-07', '2000-01-08', '2000-01-09',
               '2000-01-10'],
              dtype='datetime64[ns]', freq='D')

In [62]:
# 创建多个文件
test_nc_files_name = 0
while(test_nc_files_name<=9):
    ds.coords['y'] = ds_time_test[0+test_nc_files_name*5 : 5+test_nc_files_name*5]
    ds.to_netcdf('./test_nc_files/test_'+str(test_nc_files_name)+'.nc')
    test_nc_files_name +=1

注意下面的合并方式

* 将y（时间坐标）进行合并，在同一个y上

* tips  
一个常见的用例涉及一个分布在大量文件中的数据集，每个文件都包含大量变量。通常，这些变量中的一些需要沿着一个维度进行连接（例如"time"），而其余变量在整个数据集中是相等的（忽略浮点差异）。以下命令可以很好地与此类数据集配合使用：

In [65]:
xr.open_mfdataset('./test_nc_files/*.nc',combine='by_coords',concat_dim='y')

<xarray.Dataset>
Dimensions:  (x: 4, y: 50)
Coordinates:
    z        (x) object dask.array<chunksize=(4,), meta=np.ndarray>
  * x        (x) int64 10 20 30 40
  * y        (y) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-02-18 2000-02-19
Data variables:
    foo      (x, y) float64 dask.array<chunksize=(4, 5), meta=np.ndarray>

* 这些命令将变量沿着维度‘y’连接起来，但是只是那些已经含有‘y’维度的变量(参数data_vars='minimal',coords='minimal')，不含有维度'y'的变量将取自第一个dataset(compat='override')

In [73]:
xr.open_mfdataset('./test_nc_files/*.nc', combine='by_coords',#concat_dim="y",
                  data_vars='minimal', coords='minimal', compat='override')

<xarray.Dataset>
Dimensions:  (x: 4, y: 50)
Coordinates:
    z        (x) object dask.array<chunksize=(4,), meta=np.ndarray>
  * x        (x) int64 10 20 30 40
  * y        (y) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-02-18 2000-02-19
Data variables:
    foo      (x, y) float64 dask.array<chunksize=(4, 5), meta=np.ndarray>

* 另一种合并方式,nested，同样的坐标出现多次

In [69]:
xr.open_mfdataset('./test_nc_files/*.nc',combine='nested',concat_dim = ['y'])

<xarray.Dataset>
Dimensions:  (x: 4, y: 50)
Coordinates:
    z        (x) object dask.array<chunksize=(4,), meta=np.ndarray>
  * x        (x) int64 10 20 30 40
  * y        (y) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-02-18 2000-02-19
Data variables:
    foo      (x, y) float64 dask.array<chunksize=(4, 5), meta=np.ndarray>

mfdataset自动将多个文件合并并合并到单个xarray数据集中。建议使用xarray打开多个文件。有关并行读取的更多详细信息，请参阅Stephan Hoyer的[cobing.muti](http://xarray.pydata.org/en/stable/combining.html#combining-multi), [dask.io](http://xarray.pydata.org/en/stable/dask.html#dask-io), [blog](http://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/)。  
open_mfdataset()需要许多关键词参数kwargs，使您可以控制其行为（例如```parallel```，```combine```，```compat```，```join```，```concat_dim```）。有关更多详细信息，请参见其文档。

### 其他方式（dask）

如果```open_mfdataset（）```不满足您的需求，则可以使用其他方法。使用```dask```并行读取多个文件，修改这些数据集然后合并为一个```Dataset```的一般模式是：

In [76]:
def modify(ds):
    # modify ds here
    return ds

In [82]:
# 下面用到
import dask.delayed
from glob import glob

In [90]:
file_names = glob('./test_nc_files/*.nc')

In [113]:
# this is basically what open_mfdataset does
open_kwargs = dict(decode_cf=True, decode_times=False)
open_tasks = [dask.delayed(xr.open_dataset)(f, **open_kwargs) for f in file_names]
tasks = [dask.delayed(modify)(task) for task in open_tasks]
datasets = dask.compute(tasks)  # get a list of xarray.Datasets # 是把tuple套在list外面了
combined = xr.combine_nested(datasets[0], concat_dim='y')  # or some combination of concat, merge

In [114]:
combined

<xarray.Dataset>
Dimensions:  (x: 4, y: 50)
Coordinates:
    z        (x) object 'a' 'b' 'c' 'd'
  * x        (x) int64 10 20 30 40
  * y        (y) int64 0 1 2 3 4 0 1 2 3 4 0 1 2 3 ... 2 3 4 0 1 2 3 4 0 1 2 3 4
Data variables:
    foo      (x, y) float64 0.127 0.9667 0.2605 0.8972 ... 0.7768 0.5948 0.1376

此功能在许多情况下都可以使用，但不是很可靠。首先，它永远不会关闭文件，这意味着它将无法加载您需要加载数千个文件的文件。其次，它假定您希望每个文件中的所有数据都可以放入内存。在许多情况下，您只需要每个文件的数据的一小部分或汇总摘要。

这是有关如何纠正这些缺陷的更为复杂的示例：

In [115]:
def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds)
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
            ds.load()
            return ds

    paths = sorted(glob(files))
    datasets = [process_one_path(p) for p in paths]
    combined = xr.concat(datasets, dim)
    return combined

# here we suppose we only care about the combined mean of each file;
# you might also use indexing operations like .sel to subset datasets
combined = read_netcdfs('/all/my/files/*.nc', dim='time',
                        transform_func=lambda ds: ds.mean())

ValueError: must supply at least one object to concatenate

这种模式效果很好，并且非常健壮。我们使用了类似的代码来处理成千上万个文件，这些文件构成了100 GB的数据。（~~自夸的话~~）